# Practical session 3: Parallel behaviors and more sensing abilities

In the last practical session, we saw how to define, attach, start, stop and detach a behavior on an agent agent. We implemented three distinct behaviors: `slow_down`, `fear` and `aggression`.

In this section we will see more sensing abilities the agent is equipped with, will define new behaviors using them and will see how to deal with multiple behaviors running in parallel on the same agent. At the end of the session, we will also see how to attach those behaviors on multiple agents interacting together within a Vivarium scene.

As usual, start the server, the interface and open the simulator session:

In [ ]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

In [ ]:
start_server_and_interface(scene_name="session_3")

Wait until the interface link shows up (http://localhost:5006/run_interface) and click on it, and make sure the scene is present on your browser. You should now only see one agent in the environment. If it is the case, you can ignore this instruction and skip to the next cell.

Else, if you see two agents, it means a plotting error happened, and that simulations that shouldn't exist are still displayed. To fix it, open the `SIMULATOR` tab of the interface. Inside this tab, untick the `Hide non existing` checkbox, and tick it again. The scene should now be displayed correctly.


Create a controller that will be used to control the simulation with Python code from this jupyter notebook:

In [ ]:
controller = NotebookController()

Start the simulator:

In [ ]:
controller.run()

## Selectively detecting scene objects

To define a repertoire of interesting behaviors, we need the agent to selectively sense the proximity of different types of entities around them. For example, we might want to define a behavior for obstacle avoidance and another one for attraction towards mates. The first behavior will require sensors information about objects, whereas the second will require the proximeters to detect other agents (although there is only one agent in the scene for now, we'll add more at the end of this session). 

Since there is only one agent, let's create an alias variable to access it as in previous sessions:

In [ ]:
agent = controller.agents[0]

Let's make the agent turn on itself, and call the `sensors` function with different orientations of the agent to test it works well.

In [ ]:
# make the agent turn on itself
agent.left_motor = 0.5

First, ensure the following cell gives different values each time you call it:

In [ ]:
left, right = agent.sensors()
print(left, right)

As seen in interface, there are three types of entity in the current scene: a blue circle and a number of orange and green squares. By default in Vivarium, circle entites represent agents and square entities represent objects. Here we have two subtypes of objects: large orange ones and smaller green ones. In this sessions, we call the orange objects "obstacles" and the green objects "resources". 

We can print the names of the existing subtypes in the scene with:

In [ ]:
controller.print_subtypes_list()

Just for informaiton, the types (agent or objects), subtypes (agents, obstacles or resources) and their respective colors are defined in a scene configuration file. The configuration file of the current scene is located at [conf/scene/session_3.yaml](../../conf/scene/session_3.yaml). 

We can filter the result returned by the agent's proximeters by providing the argument `sensed_entities` to the `sensors` function. Let's detect only one type of entities (e.g obstacles). It should only return positive values when the proximeters are sensing the obstacles.

In [ ]:
left, right = agent.sensors(sensed_entities=["obstacles"])
print(left, right)

Executing the cell above will return the proximeter activations only for the `obstacles` entities (the orange squares in the scene). Give it a try by first stopping the agent wheels:

In [ ]:
agent.stop_motors()

Then move an obstacle in the detection area of the proximiters and re-executing the cell above that calls `agent.sensor(.)` to observe the change in the returned values. You can also check that the proximeter activations are not modified by other objects such as ressources (little green squares).

Note that selective sensive can be occluded by other entities, whatever their subtype. This mean that if e.g. a `resource` object is closer than any `obstacle` object in the proximiter field of view, then `agent.sensors(sensed_entities=["obstacles"])` will return `0` for that proximiter. This is somehow similar to how our own eyes sense objects: if you look at a tree but there is a wall between the tree and yourself, you won't see that tree. 

The `sensed_entities` argument requires a list of strings (`["obstacles"]` in the example above). In Python, a list is a collection of values separated by commas and surrounded by square bracket: `["obstacles"]` is therefore a list of only one element (the string `"obstacles"`), whereas `["obstacles", "agents"]` is a list of two elements (the strings `"obstacles"` and `"agents"`). 

The `sensed_entities` argument, as its name indicates, specifies the entities to be sensed by the proximeters. You can get the subtype of any entity in the scene by using the `print_infos` function, and looking at the `Subtype` field. For example, select the object with index `0`in the interface (you can do this by clicking on `0` in the `Selection` list of the `OBJECT` column on the interface). You check the subtype of the object with index `0` with: 

In [ ]:
object_idx = 0  # index of the object of interest
object = controller.objects[object_idx]  # get the object of interest
object.print_infos()  # print the object's information

If the object you selected in the interface was a green square (resp. an orange square), the subtyte printed above should `resources` (resp. obstacle). Alternatively you can also look at the value of the `Subtype` field at the bottom of the `OBJECT` column in the interface. Here the subtype is encoded as an integer, which correspond to the order in which subtypes are defined in the scene configuration file (the yaml file mentioned above), starting at `0`. 

You can also do the same for the agent: 

In [ ]:
agent.print_infos()

An agent's sensor can also detect multiple subtypes of entities:

In [ ]:
left, right = agent.sensors(sensed_entities=["resources", "obstacles"])
print(left, right)

In that case it will sense the closest entities that are of one of the indicated subtype (i.e. here the closest entities which are either resources or obstacles)

This function gives an error message if you provide a string that doesn't correspond to an existing subtype or that is spelled wrong, and ask you to select a type among the correct ones :

In [ ]:
left, right = agent.sensors(sensed_entities=["resssources"])  # typo in the entity subtype

**Q1:** Write the code printing the proximeter activations for resources and test that it works as expected by placing your agent close to those objects and verifying the returned values:

In [ ]:
# your code here

**Q2:** Define an `obstacle_avoidance` behavior. The agent has to turn in the direction opposite to the obstacles it detects, with its speed inversely proportional to the proximeter activations (the closer an obstacle, the lower the speed). 
*Tip:* it is similar to the `shyness` behavior of [Braitenberg vehicles](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0).

In [ ]:
# your code here

Remember that to test a behavior, you first have to detach all behaviors that could still be attached to the agent, then to attach the new one, that is:

In [ ]:
agent.detach_all_behaviors()
agent.attach_behavior(obstacle_avoidance)

The agent should now smoothly navigate between the obstacles in the scene. You can now detach this behavior and stop its motors.

In [ ]:
agent.detach_all_behaviors(stop_motors=True)

## Environmental dynamics

Until now, the environment in which the agent is evolving was quite static: although some objects can be pushed by the agent (e.g. the obstacles or the resources), there is nothing that appears or disappears in the environment. We are now going to see how we can generate food sources appearing at random positions in the environment and disappearing whenever a agent eats them. A food source is modeled as a `resources` object (the green squares). Before making these resources spawn, we need to do a few things before implementing a mechanism where the agent will eat ressources :

- Add `resources` to the agent's diets
- Start the eating mechanism with the controller
- Start the apparition of `resources` with the controller

### Specifying the diet of agents

First, let's handle the diet of the agent, we can check his current diet and eating range with the following code: 

In [ ]:
print(f"\nAgent eating information:")
print(f"Current agent diet: {agent.diet}")
print(f"Current agent eating range: {agent.eating_range}")

The diet of the agent is a list of entity subtypes the agent can it. As we see above the list is currently empty, which means that our agent is not able to eat anything. 

The default eating mechanism in Vivarium is pretty basic: an agent will eat an entity whose subtype is in its diet whenever that entity is at a distance lesser than the specified eating range. As seen above, the current eating range is 10. 

You can also get theses informations by using the print_infos() function, with `full_infos` set to True (but it will also print a lot of other informations you might not be interested in): 

In [ ]:
agent.print_infos(full_infos=True)

You can then manually add ressources to the agent's diet, by setting it in a list of strings, as for the `sensed_entities` argument of the `sensors` function:

In [ ]:
agent.diet = ["resources"]

If we wanted them to eat other kind of entities such as obstacles, we would have to add `obstacles` to the diet list of the agents.

Now the agent will eat resources, i.e. the green squares, whenever they are within their eating range. But to enable it we first need to start the eating mechanism as explained below.

### Start the eating mechanism

To enable the eating mechanism we need to start it on the controller with:

In [ ]:
controller.start_eating_mechanism()

From now on, whenever a resource is within the eating range of the agent, it will "eat" it. This means that the resource will disappear from the environment once eaten. The agent will also receive an internal signal indicating it has eaten something, which we will explain it later. You can test that resources indeed disappear once eaten by manually moving the agent close to a resource (either by setting motor activation in the notebook or with drag n drop in the interface).

### Start spawning of resources

Now to make this scene more interesting, we can start ressources spawn, i.e. regularly adding new resources in the environment. To do so, we need to specify an interval of simulation steps between each apparition of a resource. To estimate what this interval could be, we can use the `print_fps` method of the controller to know how many steps per seconds are executed in the simulation:

In [ ]:
controller.print_fps()

e.g. if we want a resource to appear every 2 seconds, we will set the interval to 2 * FPS.

In [ ]:
# replace by your actual fps value
fps = 30
seconds = 2
interval = seconds * fps
print(interval)

Then, start the spawning of resources with the following command:

In [ ]:
controller.start_resources_apparition(interval=interval)

In [ ]:
agent.detach_all_behaviors()
agent.attach_behavior(obstacle_avoidance)

The resources should now appear at random positions in the scene. This will be done until the maximum number of resources is reached in the environment (it is set to 12 for this session). If this is the case and the agent eats a resource somewhere, another one will appear at a random position in the scene.

You can also stop the spawning of resources with the following command:

In [ ]:
controller.stop_resources_apparition()

If you want to, you can also remove all the entities of a certain type with this command:

In [ ]:
controller.remove_entity_type("resources")

If the resource apparition function is still running, the controller will keep spawning resources even after removing them with the previous cell. When executing the previous cell it might print messages such as `Entity x already removed`, don't worry about those for now.

### Custom position of resources spawning

You can also control the position range where the resources will appear with the `position_range` parameter. This parameter is a list of 4 values: ((x_min, x_max), (y_min, y_max)) where x_min and x_max are the minimum and maximum x coordinates of the spawning area, and y_min and y_max are the minimum and maximum y coordinates of the spawning area. For example, to make resources appear only in the area between x=0 and x=50, and y=100 and y=200:

In [ ]:
# then, start the spawning apparition in a specific area
controller.start_resources_apparition(interval=interval, position_range=((0, 50), (100, 200)))

The cell above will generate resources at random 2D positions $(x, y)$ in the scene with $x\in[0, 50]$, $y\in[100, 200]$ (analyze the cell above to understand how the `position_range` argument is converted in $(x, y)$ intervals and ask us if it is not clear). You can easiely check the coordinate of an entity in the web interface.  

**Q3:** Write the code that removes the resources from the environment, and make them appear in the bottom-right corner of the scene (approximately the quarter of the environment):

In [ ]:
# your code here

You can play with this mechanism a bit, and then stop it.

In [ ]:
controller.stop_resources_apparition()

## Combining behaviors

**Q4:** Define a behavior allowing the agent to catch food sources, let's call it `foraging`. The agent has to orient itself toward food sources, with a speed proportional to the proximiter activations (the closer the food source, the higher the speed) 

- *Tip 1:* It's similar to the `aggression` behavior. 
- *Tip 2:* You already saw how to detect `obstacles` in the obstacle avoidance behavior. Here the agent will instead have to detect `resources`.

In [ ]:
# your code here

Attach and start the behavior on the agent: 

In [ ]:
# First detach previous behaviors that might still be attached to the agent
agent.detach_all_behaviors()

# Write the code to attach and start the `foraging` behavior below
agent.attach_behavior(foraging)

In [ ]:
agent.print_behaviors()

Whenever a resource is detected by the proximeters, the agent should go towards it. However, if at one point the proximeters don't detect any resource, the agent will probably stop (depending on how you have defined the behavior). The only event that could make the agent move again would be a resource that spawns within the proximeter detection area, which is not very likely to happen (and therefore quite a bad option if the survival of the agent depends on its foraging abilities). 

A solution to avoid such a blocking situation is to combine the `foraging` behavior with another one that keeps the agent in movement, as it is for example the case of the `obstacle_avoidance` behavior we have defined before. Let's attach and start the `obstacle_avoidance` behavior, but this time without detaching the previously attached `foraging` behavior:

In [ ]:
agent.attach_behavior(obstacle_avoidance)

Since we haven't detached the previous behavior, the agent is now executing two behaviors in parallel. This can be checked with:

In [ ]:
agent.print_behaviors()

which tells us that both behaviors are attached and started. In V-REP, you can see that the agent is now both foraging and avoiding obstacles. For doing so, the motor activation sent to each wheel corresponds to the average of the motor activation returned by each behavior (this averaging is implemented internally, you don't need to worry about it).

## Dealing with multiple agents

This section explains how to deal with multiple agents and how to attach different behaviors to them.

At the moment there is only 1 existing agent that we can see in the scene. But there is actually another one that is not "existing" yet. We can see it with:

In [ ]:
controller.agents

which returns a list with two elements in it, each one corresponding to an agent. The first element in the list is accessed with `controller.agent[0]` (in Pythons list indices starts at 0) and corresponds to the agent we have manipulated above (using the alias variable `agent` which was set to `controller.agent[0]` at the start of this notebook). We can check if this first agent exists with:

In [ ]:
controller.agents[0].exists

which is `True`, meaning that this agent exists (it is the one we see in the interface). Let's check it for the second agent (index `1` of the list):

In [ ]:
controller.agents[1].exists

which is `False`, meaning that this agent does not exist yet. We can make this agent exist with:

In [ ]:
controller.agents[1].exists = True

Now you should see a second agent in the interface. 

We will rename our original `agent` to `agent_0` and the new one to `agent_1` to make things clearer.

In [ ]:
agent_0 = controller.agents[0]
agent_1 = controller.agents[1]

`agent_0` is the same as the one we called `agent` before: you can see it moving the same in the interface, and also check he still has the same behaviors attached to it.

In [ ]:
agent_0.print_behaviors()

Let's change the color of the second agent to be red, so that we can differentiate it from the first one, and also add the 'resources' to its diet.

In [ ]:
agent_1.color = 'red'
agent_1.diet = ["resources"]

Now you have access to the two agents through the variables `agent_0` and `agent_1` (these variables names are arbitrary, you can choose whatever you want, e.g. `predator` and `prey`). You can attach and start behaviors on each agent independently, in the same way as you did before, simply using either the `agent_0` and `agent_1` variables instead of only `agent` one as before.

As an example, let's say we want to attach the `obstacle_avoidance` behavior we have defined above to `agent_0`, and both the `obstacle_avoidance` and the `foraging` behaviors to `agent_1`. 

In [ ]:
# detach the agent_0 behaviors and only attach the obstacle_avoidance behavior
agent_0.detach_all_behaviors()
agent_0.attach_behavior(obstacle_avoidance)
print("\nAgent_0 behaviors:")
agent_0.print_behaviors()

# attach the obstacle_avoidance and foraging behaviors to agent_1
agent_1.attach_behavior(obstacle_avoidance)
agent_1.attach_behavior(foraging)
print("\nAgent_1 behaviors:")
agent_1.print_behaviors()

Start the resources apparition mechanism again to see if `agent_1` (i.e. the red one) is able to catch the resources.

In [ ]:
controller.start_resources_apparition(interval=interval)

**Q5:** Implement the `fear` and `aggression` behaviors so that they are directed only toward the other agent, using the `sensed_entities` argument of the `sensors` method as we have seen above. Then attach both the `obstacle_avoidance` and the `aggression` behaviors to one agent, and both the `obstacle_avoidance` and the `fear` behaviors on the other. If you did it well, you should observe a simple "prey-predator" interaction, where `agent_0` tries to catch `agent_1` and `agent_1` tries to escape from `agent_0`.

First, you can detach the behaviors and stop the motors of both agents with the following cell. Using this `for` loop on the `controller.agents` and the `detach_behaviors` function enables you to detach the behaviors of all the agents at once.

In [ ]:
# your code here

Define the `fear` and `aggression` behaviors towards other agents in the cell below.

- *Tip 1:* You already saw how to define `aggression` behavior toward every entity in session 2. 
- *Tip 2:* You now want to fear and agress other agents, you might want to specificaly sense them here. The agents are of subtype `agents`.

In [ ]:
# your code here

Modify the cell below to attach the previously defined behaviors to the agents. We also directly provide lines of code to modify the aspect of the agents (size, color) and their speed.

In [ ]:
# attach the aggression and obstacle_avoidance behaviors to agent_0, and increase its diameter
# your code here
agent_0.diameter = 10.
agent_0.color = 'red'
agent_0.wheel_diameter = 3.0

# attach the fear and obstacle_avoidance behaviors to agent_1, decrease its diameter, and increase its speed
# your code here
agent_1.diameter = 4.
agent_1.color = 'cyan'
agent_1.wheel_diameter = 3.5

You should observe that `agent_0` is chasing `agent_1` when it detects it, and that `agent_1` is avoiding `agent_0`. They should be both avoiding obstacles as well. If the presence of obstacles and resources is perturbating, we can remove them from the scene with:

In [ ]:
controller.stop_resources_apparition()

controller.remove_entity_type("obstacles")
controller.remove_entity_type("resources")

## Modifying the sensor fields of view

But because the sensors of `agent_1` are only directed in the forward direction, it is really hard to avoid the `agent_0` when it comes from behind it (it cannot see it in that case). Additionally, the `agent_0` is a predator but has a pretty bad vision because it can't see very far.


We can fix this by modifying the sensors characteristics of the agents, by changing their angles and max ranges by using the following cell. 

- The `prox_dist_max` parameter is the maximum distance at which the sensors can detect entities
- The `prox_cos_min` parameter is the minimum cosine of the angle between the sensor and the entity in order to detect it

So the `prox_cos_min` value is between -1 and 1. The closer this value is to 1, the narrower the sensor field of view, and inversely for -1.

In [ ]:
# increase the range of the big agent and decrease its angle
agent_0.proxs_dist_max = 100.
agent_0.proxs_cos_min = 0.8

In [ ]:
# decrease the range of the small agent and increase its angle
agent_1.proxs_dist_max = 40.
agent_1.proxs_cos_min = -0.9

That's it for today. Don't forget to properly close the simulator before closing this notebook:

In [ ]:
controller.stop()
stop_server_and_interface(safe_mode=True)

Now that you finished session 3, you can now jump to the notebook of [session 4](session_4.ipynb).